# Age & Sex Statistics

## Import Needed Libraries

In [131]:
import pandas as pd
from openpyxl import workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import numpy as np
import os
import math

## Loading file to dataframes

List of dataframes and their proper order

In [132]:
# Creating ordered list of DFs to name each sheet coming in from workbook
df_names = ['DF_total_all','DF_male_all','DF_female_all','DF_total_whi',
            'DF_total_male_whi','DF_total_female_whi','DF_total_baa',
            'DF_total_male_baa','DF_total_female_baa','DF_total_aian',
            'DF_total_male_aian','DF_total_female_aian','DF_total_aa',
            'DF_total_male_aa','DF_total_female_aa','DF_total_nhop',
            'DF_total_male_nhop','DF_total_female_nhop','DF_total_sor',
            'DF_total_male_sor','DF_total_female_sor','DF_total_tom',
            'DF_total_male_tom','DF_total_female_tom','DF_total_hol',
            'DF_total_male_hol','DF_total_female_hol'
            ]

### Variables for column names and type

In [133]:
cols_int = ['Under 1 Year','1 Year','2 Years','3 Years','4 Years','5 Years',
            '6 Years','7 Years','8 Years','9 Years','10 Years','11 Years',
            '12 Years','13 Years','14 Years','15 Years','16 Years','17 Years',
            '18 Years','19 Years','20 Years','21 Years','22 Years','23 Years',
            '24 Years','25 Years','26 Years','27 Years','28 Years','29 Years',
            '30 Years','31 Years','32 Years','33 Years','34 Years','35 Years',
            '36 Years','37 Years','38 Years','39 Years','40 Years','41 Years',
            '42 Years','43 Years','44 Years','45 Years','46 Years','47 Years',
            '48 Years','49 Years','50 Years','51 Years','52 Years','53 Years',
            '54 Years','55 Years','56 Years','57 Years','58 Years','59 Years',
            '60 Years','61 Years','62 Years','63 Years','64 Years','65 Years',
            '66 Years','67 Years','68 Years','69 Years','70 Years','71 Years',
            '72 Years','73 Years','74 Years','75 Years','76 Years','77 Years',
            '78 Years','79 Years','80 Years','81 Years','82 Years','83 Years',
            '84 Years','85 Years','86 Years','87 Years','88 Years','89 Years',
            '90 Years','91 Years','92 Years','93 Years','94 Years','95 Years',
            '96 Years','97  Years','98  Years','99  Years','100 to 104  Years',
            '105 to 109  Years','110  Years and Over'
            ]

cols_str = ['Location', 'State', 'County', 'FIPS']

### Loading process

In [134]:
# Dictionary to store the dataframes
dfs = {}

# Get the path of the current working directory
cwd = os.getcwd()

# Construct the full path to the .xlsx file
file_path = os.path.join(cwd, '2020_agesex_data.xlsx')

# Load each sheet of the .xlsx file into a named dataframe
for name in df_names:
    df = pd.read_excel(file_path, sheet_name=name, header=None)  # Specify header=None to treat the first row as data
    df.columns = df.iloc[0]  # Set the first row as the column headers
    df = df[1:]  # Exclude the first row from the data
    df.reset_index(drop=True, inplace=True)  # Reset the index
    
    # Change column types based on column names
    for column in cols_int:
        if column in df.columns:
            df[column] = df[column].astype(int)
    
    for column in cols_str:
        if column in df.columns:
            df[column] = df[column].astype(str)
    
    dfs[name] = df

### Verifying

Print all the names, shapes, and dtypes of the dataframes loaded in

In [135]:
# Print the names, shapes, and dtypes of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for name, df in dfs.items():
    print(name)
    print("Shape:", df.shape)
    print("Dtypes:")
    for column, dtype in df.dtypes.items():
        print(f"{column}: {dtype}")
    print()

Data Frames in 'dfs':
DF_total_all
Shape: (3221, 5)
Dtypes:
Total: object
Location: object
State: object
County: object
FIPS: object

DF_male_all
Shape: (3221, 108)
Dtypes:
Total: object
Under 1 Year: int32
1 Year: int32
2 Years: int32
3 Years: int32
4 Years: int32
5 Years: int32
6 Years: int32
7 Years: int32
8 Years: int32
9 Years: int32
10 Years: int32
11 Years: int32
12 Years: int32
13 Years: int32
14 Years: int32
15 Years: int32
16 Years: int32
17 Years: int32
18 Years: int32
19 Years: int32
20 Years: int32
21 Years: int32
22 Years: int32
23 Years: int32
24 Years: int32
25 Years: int32
26 Years: int32
27 Years: int32
28 Years: int32
29 Years: int32
30 Years: int32
31 Years: int32
32 Years: int32
33 Years: int32
34 Years: int32
35 Years: int32
36 Years: int32
37 Years: int32
38 Years: int32
39 Years: int32
40 Years: int32
41 Years: int32
42 Years: int32
43 Years: int32
44 Years: int32
45 Years: int32
46 Years: int32
47 Years: int32
48 Years: int32
49 Years: int32
50 Years: int32
51 

## Building Percent to Total column
Percent to total for male & female age groups

### Grouping Dataframes with like structures

In [136]:
perc_df = ['DF_male_all','DF_female_all','DF_total_male_whi',
             'DF_total_female_whi','DF_total_male_baa','DF_total_female_baa',
             'DF_total_male_aian','DF_total_female_aian','DF_total_male_aa',
             'DF_total_female_aa','DF_total_male_nhop','DF_total_female_nhop',
             'DF_total_male_sor','DF_total_female_sor','DF_total_male_tom',
             'DF_total_female_tom','DF_total_male_hol','DF_total_female_hol'
             ]

perc_df_2 = ['DF_total_all','DF_total_whi','DF_total_baa','DF_total_aian',
             'DF_total_aa','DF_total_nhop','DF_total_sor','DF_total_tom',
             'DF_total_hol'
             ]

### Percent to total process

In [137]:
# Iterate over the dictionary of dataframes
for df_name, df in dfs.items():
    if df_name in perc_df:
        # Get the list of columns in the dataframe that match cols_int
        columns = [col for col in df.columns if col in cols_int]
        updated_columns = []

        # Reset the index to consolidate memory layout
        df.reset_index(drop=True, inplace=True)

        # Calculate the percentage values for the new column
        for col in columns:
            new_col_name = f'{col}_perc'
            updated_columns.extend([col, new_col_name])

            df[new_col_name] = [0 if total == 0 else (value / total)
                                for value, total in zip(df[col], df['Total'])]
        # Append ['Location', 'State', 'County', 'FIPS'] to updated_columns
        updated_columns.extend(['Total','Location', 'State', 'County', 'FIPS'])
        # Reorder the columns in the dataframe
        df = df[updated_columns]

        # Update the dataframe in the 'dfs' dictionary
        dfs[df_name] = df

C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3660381315.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = [0 if total == 0 else (value / total)
C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3660381315.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = [0 if total == 0 else (value / total)
C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3660381315.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

### Verifying

Print all the name,shape,# of cols, col names of the dataframes

In [138]:
# Print the shape of the dataframes in 'dfs'
print("Data Frames in 'perc_df':")
for df_name, df in dfs.items():
    if df_name in perc_df:
        print("Dataframe Name:", df_name)
        print("Shape:", df.shape)
        print(f"Number of Columns: {len(df.columns)}")
        print("Columns:", df.columns.tolist())

Data Frames in 'perc_df':
Dataframe Name: DF_male_all
Shape: (3221, 211)
Number of Columns: 211
Columns: ['Under 1 Year', 'Under 1 Year_perc', '1 Year', '1 Year_perc', '2 Years', '2 Years_perc', '3 Years', '3 Years_perc', '4 Years', '4 Years_perc', '5 Years', '5 Years_perc', '6 Years', '6 Years_perc', '7 Years', '7 Years_perc', '8 Years', '8 Years_perc', '9 Years', '9 Years_perc', '10 Years', '10 Years_perc', '11 Years', '11 Years_perc', '12 Years', '12 Years_perc', '13 Years', '13 Years_perc', '14 Years', '14 Years_perc', '15 Years', '15 Years_perc', '16 Years', '16 Years_perc', '17 Years', '17 Years_perc', '18 Years', '18 Years_perc', '19 Years', '19 Years_perc', '20 Years', '20 Years_perc', '21 Years', '21 Years_perc', '22 Years', '22 Years_perc', '23 Years', '23 Years_perc', '24 Years', '24 Years_perc', '25 Years', '25 Years_perc', '26 Years', '26 Years_perc', '27 Years', '27 Years_perc', '28 Years', '28 Years_perc', '29 Years', '29 Years_perc', '30 Years', '30 Years_perc', '31 Yea

## Building Weighted Average Column

Removing modified dataframes from dict dfs for further processing

In [139]:
# Initialize the new dictionary 'dfs_perc'
dfs_perc = {}

# Iterate over the dataframe names in 'perc_df' list
for df_name in perc_df:
    # Check if the dataframe name exists in 'dfs' dictionary
    if df_name in dfs:
        # Move the matching dataframe from 'dfs' to 'dfs_perc'
        dfs_perc[df_name] = dfs.pop(df_name)

### Verifying

In [140]:
# Printing list of both dataframe dictionaries to verify last step

# We should have 9 in dict 'dfs' and 18 in new dict 'perc_df'

print("Dictionary 'dfs':")
print(len(dfs.keys()))
print(list(dfs.keys()))

print("\nDictionary 'perc_df':")
print(len(dfs_perc.keys()))
print(list(dfs_perc.keys()))

Dictionary 'dfs':
9
['DF_total_all', 'DF_total_whi', 'DF_total_baa', 'DF_total_aian', 'DF_total_aa', 'DF_total_nhop', 'DF_total_sor', 'DF_total_tom', 'DF_total_hol']

Dictionary 'perc_df':
18
['DF_male_all', 'DF_female_all', 'DF_total_male_whi', 'DF_total_female_whi', 'DF_total_male_baa', 'DF_total_female_baa', 'DF_total_male_aian', 'DF_total_female_aian', 'DF_total_male_aa', 'DF_total_female_aa', 'DF_total_male_nhop', 'DF_total_female_nhop', 'DF_total_male_sor', 'DF_total_female_sor', 'DF_total_male_tom', 'DF_total_female_tom', 'DF_total_male_hol', 'DF_total_female_hol']


### Variables for columns to drop to new dictionary

In [141]:
columns_to_drop = ['Under 1 Year_perc','1 Year_perc','2 Years_perc',
                  '3 Years_perc','4 Years_perc','5 Years_perc','6 Years_perc',
                  '7 Years_perc','8 Years_perc','9 Years_perc','10 Years_perc',
                  '11 Years_perc','12 Years_perc','13 Years_perc',
                  '14 Years_perc','15 Years_perc','16 Years_perc',
                  '17 Years_perc','18 Years_perc','19 Years_perc',
                  '20 Years_perc','21 Years_perc','22 Years_perc',
                  '23 Years_perc','24 Years_perc','25 Years_perc',
                  '26 Years_perc','27 Years_perc','28 Years_perc',
                  '29 Years_perc','30 Years_perc','31 Years_perc',
                  '32 Years_perc','33 Years_perc','34 Years_perc',
                  '35 Years_perc','36 Years_perc','37 Years_perc',
                  '38 Years_perc','39 Years_perc','40 Years_perc',
                  '41 Years_perc','42 Years_perc','43 Years_perc',
                  '44 Years_perc','45 Years_perc','46 Years_perc',
                  '47 Years_perc','48 Years_perc','49 Years_perc',
                  '50 Years_perc','51 Years_perc','52 Years_perc',
                  '53 Years_perc','54 Years_perc','55 Years_perc',
                  '56 Years_perc','57 Years_perc','58 Years_perc',
                  '59 Years_perc','60 Years_perc','61 Years_perc',
                  '62 Years_perc','63 Years_perc','64 Years_perc',
                  '65 Years_perc','66 Years_perc','67 Years_perc',
                  '68 Years_perc','69 Years_perc','70 Years_perc',
                  '71 Years_perc','72 Years_perc','73 Years_perc',
                  '74 Years_perc','75 Years_perc','76 Years_perc',
                  '77 Years_perc','78 Years_perc','79 Years_perc',
                  '80 Years_perc','81 Years_perc','82 Years_perc',
                  '83 Years_perc','84 Years_perc','85 Years_perc',
                  '86 Years_perc','87 Years_perc','88 Years_perc',
                  '89 Years_perc','90 Years_perc','91 Years_perc',
                  '92 Years_perc','93 Years_perc','94 Years_perc',
                  '95 Years_perc','96 Years_perc','97  Years_perc',
                  '98  Years_perc','99  Years_perc','100 to 104  Years_perc',
                  '105 to 109  Years_perc','110  Years and Over_perc','Total',
                  'Location','State','County','FIPS'
                  ]

### Dropping process

In [142]:
dropped_columns = {}

# Iterate over the dataframes in 'dfs_perc'
for df_name, df in dfs_perc.items():
    # Store the dropped columns' data
    dropped_columns[df_name] = df[columns_to_drop]

    # Drop the specified columns from each dataframe
    dfs_perc[df_name] = df.drop(columns_to_drop, axis=1)

#### Verifying

In [143]:
print("Dictionary 'dfs_perc':")
for df_name, df in dfs_perc.items():
    print(f"DataFrame '{df_name}' column names:")
    print(list(df.columns))

print("\nDictionary 'dropped_columns':")
for df_name, df in dropped_columns.items():
    print(f"DataFrame '{df_name}' column names:")
    print(list(df.columns))

Dictionary 'dfs_perc':
DataFrame 'DF_male_all' column names:
['Under 1 Year', '1 Year', '2 Years', '3 Years', '4 Years', '5 Years', '6 Years', '7 Years', '8 Years', '9 Years', '10 Years', '11 Years', '12 Years', '13 Years', '14 Years', '15 Years', '16 Years', '17 Years', '18 Years', '19 Years', '20 Years', '21 Years', '22 Years', '23 Years', '24 Years', '25 Years', '26 Years', '27 Years', '28 Years', '29 Years', '30 Years', '31 Years', '32 Years', '33 Years', '34 Years', '35 Years', '36 Years', '37 Years', '38 Years', '39 Years', '40 Years', '41 Years', '42 Years', '43 Years', '44 Years', '45 Years', '46 Years', '47 Years', '48 Years', '49 Years', '50 Years', '51 Years', '52 Years', '53 Years', '54 Years', '55 Years', '56 Years', '57 Years', '58 Years', '59 Years', '60 Years', '61 Years', '62 Years', '63 Years', '64 Years', '65 Years', '66 Years', '67 Years', '68 Years', '69 Years', '70 Years', '71 Years', '72 Years', '73 Years', '74 Years', '75 Years', '76 Years', '77 Years', '78 Year

### Variables for columns names

In [144]:
cols_int = ['Under 1 Year','1 Year','2 Years','3 Years','4 Years','5 Years',
            '6 Years','7 Years','8 Years','9 Years','10 Years','11 Years',
            '12 Years','13 Years','14 Years','15 Years','16 Years','17 Years',
            '18 Years','19 Years','20 Years','21 Years','22 Years','23 Years',
            '24 Years','25 Years','26 Years','27 Years','28 Years','29 Years',
            '30 Years','31 Years','32 Years','33 Years','34 Years','35 Years',
            '36 Years','37 Years','38 Years','39 Years','40 Years','41 Years',
            '42 Years','43 Years','44 Years','45 Years','46 Years','47 Years',
            '48 Years','49 Years','50 Years','51 Years','52 Years','53 Years',
            '54 Years','55 Years','56 Years','57 Years','58 Years','59 Years',
            '60 Years','61 Years','62 Years','63 Years','64 Years','65 Years',
            '66 Years','67 Years','68 Years','69 Years','70 Years','71 Years',
            '72 Years','73 Years','74 Years','75 Years','76 Years','77 Years',
            '78 Years','79 Years','80 Years','81 Years','82 Years','83 Years',
            '84 Years','85 Years','86 Years','87 Years','88 Years','89 Years',
            '90 Years','91 Years','92 Years','93 Years','94 Years','95 Years',
            '96 Years','97  Years','98  Years','99  Years','100 to 104  Years',
            '105 to 109  Years','110  Years and Over'
            ]
cols_mod = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
            25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,
            47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,
            69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,
            91,92,93,94,95,96,97,98,99,102,107,110
            ] 


### Column change process

In [145]:
# Iterate over the dataframes in 'dfs_perc'
for df_name, df in dfs_perc.items():
    # Change the column names to the list of integers
    df.columns = cols_mod
    
    # Convert the column type to integer
    df[cols_mod] = df[cols_mod].astype(int)

#### Verifying

In [146]:
# Iterate over the dataframes in 'dfs_perc'
for df_name, df in dfs_perc.items():
    print(f"DataFrame '{df_name}' column names:")
    print(list(df.columns))
    print(f"DataFrame '{df_name}' column types:")
    print(df.dtypes)
    print()

DataFrame 'DF_male_all' column names:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 107, 110]
DataFrame 'DF_male_all' column types:
0      int32
1      int32
2      int32
3      int32
4      int32
       ...  
98     int32
99     int32
102    int32
107    int32
110    int32
Length: 103, dtype: object

DataFrame 'DF_female_all' column names:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 

## Finding weighted Average

In [147]:
# Iterate over each key-value pair in dfs_perc
for df_name, df in dfs_perc.items():
    # Get the column names as an array of integers
    values = np.array(df.columns, dtype=int)

    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Get the row entries as an array of integers
        weights = np.array(row.values, dtype=int)

        # Perform element-wise multiplication of values and weights
        weighted_values = values * weights

        # Sum the products of the multiplications
        weighted_sum = np.sum(weighted_values)

        # Sum all items in the weights array
        weights_sum = np.sum(weights)

        # Calculate the weighted average
        weighted_average = weighted_sum / weights_sum

        # Append the weighted average to the results list
        results.append(weighted_average)

    # Add the 'Average_Age' column to the dataframe
    df['Average_Age'] = results

    # Reset the index of the dataframe
    df.reset_index(drop=True, inplace=True)

C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3227147353.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Average_Age'] = results
C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3227147353.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Average_Age'] = results
C:\Users\Derek\AppData\Local\Temp\ipykernel_30404\3227147353.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at onc

### Verifying

In [148]:
# Iterate over the dictionaries in 'dropped_string_columns'
for df_name, df_dict in dfs_perc.items():
    print(f"DataFrame '{df_name}' column names:")
    print(list(df_dict.keys()))
    print()

DataFrame 'DF_male_all' column names:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 107, 110, 'Average_Age']

DataFrame 'DF_female_all' column names:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 107, 110, 'Average_Age']

DataFrame 'DF_total_male_whi' column names:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

verifying values in column of specific dataframe in 'dfs_perc'

In [149]:
# Check if 'DF_male_all' exists in 'dfs_perc' dictionary
if 'DF_male_all' in dfs_perc:
    print("DataFrame 'DF_male_all':")
    print(dfs_perc['DF_male_all'])
else:
    print("DataFrame 'DF_male_all' does not exist in 'dfs_perc' dictionary.")

DataFrame 'DF_male_all':
        0    1    2    3    4    5    6    7    8    9  ...  94  95  96  97  \
0     437  450  511  484  513  525  576  559  549  564  ...   6   8   3   3   
1      70   70   84   91   80   90  116   67   97   81  ...   0   2   0   0   
2      72   80   83   60   82   78   69   84   67   90  ...   3   1   4   3   
3     157  203  182  171  192  161  202  186  158  184  ...   4   7   0   4   
4      64   51   65   60   62   79   85   71   76   71  ...   2   0   2   0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ..  ..  ..  ..   
3216   98   86  120  100  109  120  115   96  114   96  ...   3   2   4   0   
3217  183  124  170  184  175  185  185  160  188  197  ...   5   5   2   1   
3218  190  218  223  195  287  247  211  223  237  185  ...  26   6   3   6   
3219   20   10    4   14    9   10   16   17   18   11  ...   0   0   0   0   
3220  593  582  669  639  691  704  658  650  687  618  ...  13   5   3   9   

      98  99  102  107  11

## Joining Dataframes

In [150]:
# Iterate over the dataframes in 'dfs_perc'
for df_name, df_perc in dfs_perc.items():
    # Get the corresponding dataframe from 'dropped_columns'
    df_dropped = dropped_columns[df_name]

    # Concatenate the dataframes width-wise
    joined_df = pd.concat([df_perc, df_dropped], axis=1)

    # Update the 'dfs_perc' dataframe in place with the joined dataframe
    dfs_perc[df_name] = joined_df

    # Reset the index of the dataframe
    dfs_perc[df_name].reset_index(drop=True, inplace=True)

### Verifying

In [151]:
# Print the names, shapes, and dtypes of the dataframes in 'dfs_perc'
print("Data Frames in 'dfs_perc':")
for name, df in dfs_perc.items():
    print(name)
    print("Shape:", df.shape)
    print("Dtypes:")
    for column, dtype in df.dtypes.items():
        print(f"{column}: {dtype}")
    print()

Data Frames in 'dfs_perc':
DF_male_all
Shape: (3221, 212)
Dtypes:
0: int32
1: int32
2: int32
3: int32
4: int32
5: int32
6: int32
7: int32
8: int32
9: int32
10: int32
11: int32
12: int32
13: int32
14: int32
15: int32
16: int32
17: int32
18: int32
19: int32
20: int32
21: int32
22: int32
23: int32
24: int32
25: int32
26: int32
27: int32
28: int32
29: int32
30: int32
31: int32
32: int32
33: int32
34: int32
35: int32
36: int32
37: int32
38: int32
39: int32
40: int32
41: int32
42: int32
43: int32
44: int32
45: int32
46: int32
47: int32
48: int32
49: int32
50: int32
51: int32
52: int32
53: int32
54: int32
55: int32
56: int32
57: int32
58: int32
59: int32
60: int32
61: int32
62: int32
63: int32
64: int32
65: int32
66: int32
67: int32
68: int32
69: int32
70: int32
71: int32
72: int32
73: int32
74: int32
75: int32
76: int32
77: int32
78: int32
79: int32
80: int32
81: int32
82: int32
83: int32
84: int32
85: int32
86: int32
87: int32
88: int32
89: int32
90: int32
91: int32
92: int32
93: int32
94: 

Selecting a specific dataframe to verify information has migrated correctly

In [152]:
# Temp option to see all columns
with pd.option_context('display.max_columns', None):
    print(dfs_perc['DF_male_all'])

        0    1    2    3    4    5    6    7    8    9   10   11   12   13  \
0     437  450  511  484  513  525  576  559  549  564  573  595  594  597   
1      70   70   84   91   80   90  116   67   97   81   97   83   85  113   
2      72   80   83   60   82   78   69   84   67   90   82   79   84  102   
3     157  203  182  171  192  161  202  186  158  184  211  220  208  213   
4      64   51   65   60   62   79   85   71   76   71   78   85   55   92   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3216   98   86  120  100  109  120  115   96  114   96  114  113  127  117   
3217  183  124  170  184  175  185  185  160  188  197  189  181  227  181   
3218  190  218  223  195  287  247  211  223  237  185  228  244  232  248   
3219   20   10    4   14    9   10   16   17   18   11   12   18   18   19   
3220  593  582  669  639  691  704  658  650  687  618  670  700  755  720   

       14   15   16   17   18   19   20   21   22   23   24   2

## Joining Dictionaries

In [153]:
# Merge the dictionaries 'dfs' and 'dfs_perc' into 'dfs' in the order of 'df_names'
for df_name in df_names:
    if df_name in dfs_perc:
        if df_name in dfs:
            df_merged = pd.concat([dfs[df_name], dfs_perc[df_name]], axis=1)
            dfs[df_name] = df_merged
        else:
            dfs[df_name] = dfs_perc[df_name]

### Verifying

Print all the count,name,shape,# of cols, col names of the dataframes

In [ ]:
# Print the number of dataframes in the 'dfs' dictionary
print("Number of Dataframes in 'dfs':", len(dfs))

# Print the shape of the dataframes in 'dfs'
print("Data Frames in 'dfs':")
for df_name, df in dfs.items():
    print("Dataframe Name:", df_name)
    print("Shape:", df.shape)
    print(f"Number of Columns: {len(df.columns)}")
    print("Columns:", df.columns.tolist())

## Export to excel

Create workbook and export wanted dataframes to excel as individual sheets.

In [156]:
# Define the file name
file_name = '2020_agesex_statistics.xlsx'

# Get the file path in the current working directory
file_path = os.path.join(os.getcwd(), file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the existing Excel file
    excel_file = pd.read_excel(file_path, engine='openpyxl')

    # Create a new ExcelWriter object using the existing file
    writer = pd.ExcelWriter(file_path, engine='openpyxl', if_sheet_exists='replace', mode='a')

    # Iterate through the dataframes in dfs
    for df_name, df in dfs.items():  # Use 'dfs.items()' to get the name (key) and dataframe (value)
        # Get the name of the dataframe
        name = df_name

        # Write each dataframe to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=name, index=False)

    # Close the writer
    writer.close()

else:
    # Create a new workbook
    writer = pd.ExcelWriter(file_path, engine='openpyxl')

    # Iterate through the dataframes in dfs
    for df_name, df in dfs.items():  # Use 'dfs.items()' to get the name (key) and dataframe (value)
        # Get the name of the dataframe
        name = df_name

        # Write each dataframe to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=name, index=False)

    # Close the writer
    writer.close()

BadZipFile: File is not a zip file